# Setup Authentication
Import required libraries and set up environment variables for Spotify API credentials

In [18]:
# Import required libraries
import os
from dotenv import load_dotenv
import requests
from requests.auth import HTTPBasicAuth
import json
from pprint import pprint

# Load environment variables
load_dotenv()

# Get credentials
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

# Test Access Token Generation
Test the authentication process and token generation using client credentials flow

In [20]:
# Test Access Token Generation

def get_masked_token(token):
    """Return masked version of token showing only first/last 4 chars"""
    if not token:
        return None
    return f"{token[:4]}...{token[-4:]}"

# Define the authentication URL for Spotify API
auth_url = "https://accounts.spotify.com/api/token"

# Make a POST request to the authentication URL with client credentials
auth_response = requests.post(
    auth_url,
    data={"grant_type": "client_credentials"},
    auth=HTTPBasicAuth(client_id, client_secret)
)

# Check if the request was successful
if auth_response.status_code == 200:
    access_token = auth_response.json().get("access_token")
    masked_token = get_masked_token(access_token)
    print(f"Access Token: {masked_token}")
else:
    # Print the error if the request failed
    print(f"Failed to retrieve access token: {auth_response.status_code}")
    print(auth_response.json())

Access Token: BQDG...ss4w


# Make Sample API Requests
Test different API endpoints with the access token and verify the response structure

In [21]:
# Make Sample API Requests

# Define the base URL for Spotify API
base_url = "https://api.spotify.com/v1"

# Define the headers with the access token for authorization
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Make a sample request to get new releases
new_releases_url = f"{base_url}/browse/new-releases"
new_releases_response = requests.get(new_releases_url, headers=headers)

# Check if the request was successful
if new_releases_response.status_code == 200:
    # Parse and print the JSON response
    new_releases_data = new_releases_response.json()
    print("New Releases:")
    for album in new_releases_data['albums']['items']:
        print(f"Album: {album['name']}, Artist: {album['artists'][0]['name']}")
else:
    # Print the error if the request failed
    print(f"Failed to retrieve new releases: {new_releases_response.status_code}")
    print(new_releases_response.json())

# Make a sample request to get featured playlists
featured_playlists_url = f"{base_url}/browse/featured-playlists"
featured_playlists_response = requests.get(featured_playlists_url, headers=headers)

# Check if the request was successful
if featured_playlists_response.status_code == 200:
    # Parse and print the JSON response
    featured_playlists_data = featured_playlists_response.json()
    print("\nFeatured Playlists:")
    for playlist in featured_playlists_data['playlists']['items']:
        print(f"Playlist: {playlist['name']}, Description: {playlist['description']}")
else:
    # Print the error if the request failed
    print(f"Failed to retrieve featured playlists: {featured_playlists_response.status_code}")
    print(featured_playlists_response.json())

# Make a sample request to get a specific artist's information
artist_id = "1vCWHaC5f2uS3yhpwWbIA6"  # Example artist ID (Avicii)
artist_url = f"{base_url}/artists/{artist_id}"
artist_response = requests.get(artist_url, headers=headers)

# Check if the request was successful
if artist_response.status_code == 200:
    # Parse and print the JSON response
    artist_data = artist_response.json()
    print(f"\nArtist Information:\nName: {artist_data['name']}\nGenres: {', '.join(artist_data['genres'])}\nFollowers: {artist_data['followers']['total']}")
else:
    # Print the error if the request failed
    print(f"Failed to retrieve artist information: {artist_response.status_code}")
    print(artist_response.json())

New Releases:
Album: Le Clique: Vida Rockstar (X), Artist: JHAYCO
Album: LA PANTERA NEGRA, Artist: Myke Towers
Album: 166, Artist: Milo j
Album: Torii Yama, Artist: DELLAFUENTE
Album: Quién es Dei V?, Artist: Dei V
Album: LAGRIMAS DE UN MALEANTE, Artist: Omar Montes
Album: EL TIBURÓN, Artist: Rvfv
Album: EL ÚLTIMO BAILE, Artist: Trueno
Album: HIT ME HARD AND SOFT, Artist: Billie Eilish
Album: Radical Optimism, Artist: Dua Lipa
Album: SAKURA, Artist: SAIKO
Album: a new star (1 9 9 3), Artist: Rels B
Album: THE TORTURED POETS DEPARTMENT, Artist: Taylor Swift
Album: Insomnio, Artist: Abraham Mateo
Album: Las Mujeres Ya No Lloran, Artist: Shakira
Album: GRX, Artist: Lola Indigo
Album: La Joia, Artist: Bad Gyal
Album: FERXXOCALIPSIS, Artist: Feid
Album: .mp3, Artist: Emilia
Album: LVEU: VIVE LA TUYA...NO LA MIA, Artist: Myke Towers
Failed to retrieve featured playlists: 404
{'error': {'status': 404, 'message': 'Not Found'}}

Artist Information:
Name: Avicii
Genres: edm
Followers: 23316961


# Handle API Response Data
Parse and process the API response data, including error handling

In [22]:
# Handle API Response Data

# Function to handle API response data
def handle_api_response(response):
    """
    Handle the API response data, including error handling.
    
    Parameters:
    response (requests.Response): The response object from the API request.
    
    Returns:
    dict: The parsed JSON data if the request was successful.
    None: If the request failed.
    """
    if response.status_code == 200:
        try:
            # Parse the JSON response
            data = response.json()
            return data
        except ValueError:
            print("Error: Unable to parse JSON response.")
            return None
    else:
        print(f"Error: Request failed with status code {response.status_code}")
        try:
            error_data = response.json()
            print("Error details:", error_data)
        except ValueError:
            print("Error: Unable to parse error response.")
        return None

# Example usage of the handle_api_response function
# Handle new releases response
new_releases_data = handle_api_response(new_releases_response)
if new_releases_data:
    print("New Releases:")
    for album in new_releases_data['albums']['items']:
        print(f"Album: {album['name']}, Artist: {album['artists'][0]['name']}")

# Handle featured playlists response
featured_playlists_data = handle_api_response(featured_playlists_response)
if featured_playlists_data:
    print("\nFeatured Playlists:")
    for playlist in featured_playlists_data['playlists']['items']:
        print(f"Playlist: {playlist['name']}, Description: {playlist['description']}")

# Handle artist information response
artist_data = handle_api_response(artist_response)
if artist_data:
    print(f"\nArtist Information:\nName: {artist_data['name']}\nGenres: {', '.join(artist_data['genres'])}\nFollowers: {artist_data['followers']['total']}")

New Releases:
Album: Le Clique: Vida Rockstar (X), Artist: JHAYCO
Album: LA PANTERA NEGRA, Artist: Myke Towers
Album: 166, Artist: Milo j
Album: Torii Yama, Artist: DELLAFUENTE
Album: Quién es Dei V?, Artist: Dei V
Album: LAGRIMAS DE UN MALEANTE, Artist: Omar Montes
Album: EL TIBURÓN, Artist: Rvfv
Album: EL ÚLTIMO BAILE, Artist: Trueno
Album: HIT ME HARD AND SOFT, Artist: Billie Eilish
Album: Radical Optimism, Artist: Dua Lipa
Album: SAKURA, Artist: SAIKO
Album: a new star (1 9 9 3), Artist: Rels B
Album: THE TORTURED POETS DEPARTMENT, Artist: Taylor Swift
Album: Insomnio, Artist: Abraham Mateo
Album: Las Mujeres Ya No Lloran, Artist: Shakira
Album: GRX, Artist: Lola Indigo
Album: La Joia, Artist: Bad Gyal
Album: FERXXOCALIPSIS, Artist: Feid
Album: .mp3, Artist: Emilia
Album: LVEU: VIVE LA TUYA...NO LA MIA, Artist: Myke Towers
Error: Request failed with status code 404
Error details: {'error': {'status': 404, 'message': 'Not Found'}}

Artist Information:
Name: Avicii
Genres: edm
Follow

# Test Pagination and Offset
Implement and test pagination logic with different offset values

In [23]:
# Test Pagination and Offset

# Define a function to get new releases with pagination
def get_new_releases_with_offset(offset=0, limit=10):
    """
    Get new releases from Spotify API with pagination.
    
    Parameters:
    offset (int): The index of the first item to return.
    limit (int): The maximum number of items to return.
    
    Returns:
    dict: The parsed JSON data if the request was successful.
    None: If the request failed.
    """
    new_releases_url = f"{base_url}/browse/new-releases?offset={offset}&limit={limit}"
    response = requests.get(new_releases_url, headers=headers)
    return handle_api_response(response)

# Test the function with different offset values
offset_values = [0, 10, 20, 30]
for offset in offset_values:
    print(f"\nNew Releases with offset {offset}:")
    new_releases_data = get_new_releases_with_offset(offset=offset)
    if new_releases_data:
        for album in new_releases_data['albums']['items']:
            print(f"Album: {album['name']}, Artist: {album['artists'][0]['name']}")


New Releases with offset 0:
Album: Le Clique: Vida Rockstar (X), Artist: JHAYCO
Album: LA PANTERA NEGRA, Artist: Myke Towers
Album: 166, Artist: Milo j
Album: Torii Yama, Artist: DELLAFUENTE
Album: Quién es Dei V?, Artist: Dei V
Album: LAGRIMAS DE UN MALEANTE, Artist: Omar Montes
Album: EL TIBURÓN, Artist: Rvfv
Album: EL ÚLTIMO BAILE, Artist: Trueno
Album: HIT ME HARD AND SOFT, Artist: Billie Eilish
Album: Radical Optimism, Artist: Dua Lipa

New Releases with offset 10:
Album: SAKURA, Artist: SAIKO
Album: a new star (1 9 9 3), Artist: Rels B
Album: THE TORTURED POETS DEPARTMENT, Artist: Taylor Swift
Album: Insomnio, Artist: Abraham Mateo
Album: Las Mujeres Ya No Lloran, Artist: Shakira
Album: GRX, Artist: Lola Indigo
Album: La Joia, Artist: Bad Gyal
Album: FERXXOCALIPSIS, Artist: Feid
Album: .mp3, Artist: Emilia
Album: LVEU: VIVE LA TUYA...NO LA MIA, Artist: Myke Towers

New Releases with offset 20:
Album: nadie sabe lo que va a pasar mañana, Artist: Bad Bunny
Album: Poquito a Poquito

In [24]:
def search_tracks_by_year(year, offset=0, limit=10):
    """Search tracks by year and return formatted results"""
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    params = {
        'q': f'year:{year}',
        'type': 'track',
        'limit': limit,
        'offset': offset
    }
    
    try:
        response = requests.get(
            'https://api.spotify.com/v1/search',
            headers=headers,
            params=params
        )
        
        if response.status_code == 200:
            data = response.json()
            tracks = []
            
            for track in data['tracks']['items']:
                track_info = {
                    'name': track['name'],
                    'artist': track['artists'][0]['name'],
                    'album': track['album']['name'],
                    'release_date': track['album']['release_date'],
                    'preview_url': track.get('preview_url')
                }
                tracks.append(track_info)
                
            return tracks
            
        else:
            return f"Error: {response.status_code}"
            
    except Exception as e:
        return f"Error: {str(e)}"

def format_spotify_tracks(data):
    """Format track data for display"""
    formatted_tracks = []
    
    # Handle if already formatted list
    if isinstance(data, list):
        return data
        
    # Handle raw API response
    if isinstance(data, dict) and 'tracks' in data:
        tracks = data['tracks']['items']
    else:
        return []
    
    for track in tracks:
        if track.get('preview_url'):
            formatted_tracks.append({
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'album': track['album']['name'],
                'release_date': track['album']['release_date'],
                'preview_url': track['preview_url'],
                'popularity': track.get('popularity', 0)
            })
    
    return sorted(formatted_tracks, key=lambda x: x['popularity'], reverse=True)

# Test
test_year = 2020
results = search_tracks_by_year(test_year)
tracks = format_spotify_tracks(results)

# Display
print(f"\nTracks from {test_year}:")
print("-" * 50)
for track in tracks:
    print(f"🎵 {track['name']}")
    print(f"👤 {track['artist']}")
    print(f"💿 {track['album']}")
    print(f"📅 {track['release_date']}")
    print("-" * 50)


Tracks from 2020:
--------------------------------------------------
🎵 El Fin del Mundo
👤 La La Love You
💿 La la Love You Bonus
📅 2020-12-11
--------------------------------------------------
🎵 La Curiosidad
👤 Jay Wheeler
💿 Platonicos
📅 2020-06-12
--------------------------------------------------
🎵 Hace Mucho Tiempo
👤 Ele A El Dominio
💿 Hace Mucho Tiempo
📅 2020-03-06
--------------------------------------------------
🎵 Piensan
👤 Myke Towers
💿 Easy Money Baby
📅 2020-01-24
--------------------------------------------------
🎵 Tú
👤 Myke Towers
💿 Easy Money Baby
📅 2020-01-24
--------------------------------------------------
🎵 Dile a El
👤 Rauw Alejandro
💿 Afrodisíaco
📅 2020-11-13
--------------------------------------------------
🎵 Extasy
👤 Myke Towers
💿 Para Mi Ex
📅 2020-12-15
--------------------------------------------------
🎵 3 Am
👤 Eladio Carrion
💿 Sauce Boyz
📅 2020-01-31
--------------------------------------------------
🎵 Relación Rota
👤 Myke Towers
💿 Easy Money Baby
📅 2020-01-24
-